In [5]:
%%capture
! pip install investpy

In [1]:
import pandas as pd
import numpy as np
import re

In [8]:
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import seaborn as sns

In [83]:
import logging
import os
import sys

In [6]:
import investpy

In [84]:
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
PATH_MAIN = os.path.join(ROOT, 'My Drive/DS/Extra Year/BDOS/scripts')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
# Prepare Logging:
PATH_LOG = os.path.join(PATH_MAIN, 'runner/log/my_logging.log')
logFormatter = logging.Formatter("%(asctime)s [%(levelname)s]  %(message)s")
rootLogger = logging.getLogger()
fileHandler = logging.FileHandler(PATH_LOG, mode='w')
fileHandler.setFormatter(logFormatter)
rootLogger.addHandler(fileHandler)
consoleHandler = logging.StreamHandler(sys.stdout)
consoleHandler.setFormatter(logFormatter)
rootLogger.addHandler(consoleHandler)
rootLogger.setLevel(logging.INFO)

In [95]:
# Custom Libraries:
from importlib.machinery import SourceFileLoader
preprocessing = SourceFileLoader('somelib', os.path.join(PATH_MAIN, 'lib/preprocessing.py')).load_module()


# Get Data


In [96]:
START = '01/01/2019'
END = '31/12/2021'

### 1. Get Target (BTC)

In [93]:
target_df = investpy.get_crypto_historical_data(crypto='bitcoin', from_date=START, to_date=END)

In [94]:
logging.info(target_df.head().to_string())

INFO:root:              Open    High     Low   Close  Volume Currency
Date                                                       
2019-01-01  3709.5  3814.3  3664.4  3809.4  469107      USD
2019-01-02  3809.7  3894.8  3768.1  3873.8  554471      USD
2019-01-03  3874.1  3875.8  3753.0  3780.1  450319      USD
2019-01-04  3780.5  3823.9  3720.0  3802.7  488698      USD
2019-01-05  3802.8  3846.7  3769.0  3785.4  468449      USD


2022-04-04 16:02:51,066 [INFO]                Open    High     Low   Close  Volume Currency
Date                                                       
2019-01-01  3709.5  3814.3  3664.4  3809.4  469107      USD
2019-01-02  3809.7  3894.8  3768.1  3873.8  554471      USD
2019-01-03  3874.1  3875.8  3753.0  3780.1  450319      USD
2019-01-04  3780.5  3823.9  3720.0  3802.7  488698      USD
2019-01-05  3802.8  3846.7  3769.0  3785.4  468449      USD
2022-04-04 16:02:51,066 [INFO]                Open    High     Low   Close  Volume Currency
Date                                                       
2019-01-01  3709.5  3814.3  3664.4  3809.4  469107      USD
2019-01-02  3809.7  3894.8  3768.1  3873.8  554471      USD
2019-01-03  3874.1  3875.8  3753.0  3780.1  450319      USD
2019-01-04  3780.5  3823.9  3720.0  3802.7  488698      USD
2019-01-05  3802.8  3846.7  3769.0  3785.4  468449      USD
2022-04-04 16:02:51,066 [INFO]                Open    High     Low   Close  Volume Currency
Date

### 2. Get Other Cryptos Prices:

In [79]:
# TOP 10 traded cryptos (based on total volume):
all_crypto = investpy.crypto.get_cryptos_overview()
#all_crypto = all_crypto.loc[(all_crypto.name!='Binance USD') & (all_crypto.name!='BNB'), ]
all_crypto.total_volume = all_crypto.total_volume .apply( lambda x: np.float16(x[:-1]) )
crypto_names = all_crypto.sort_values(by='total_volume', ascending=False).name.iloc[:10].values

crypto_names

array(['Tether', 'Bitcoin', 'Ethereum', 'Binance USD', 'Bitcoin Cash',
       'USD Coin', 'Solana', 'BNB', 'Zilliqa', 'Terra'], dtype=object)

In [80]:
crypto_df = pd.DataFrame()
# Get data:
for name in crypto_names:
  try:
    df = investpy.get_crypto_historical_data(crypto=name, from_date=START, to_date=END)
    crypto_df = pd.concat([crypto_df, df.Close], axis=1)
  except:
    logging.error(name)


ERROR:root:Binance USD
ERROR:root:BNB


In [53]:
crypto_df

,Close,Close,Close,Close,Close,Close,Close,Close
2019-01-01,1.0063,3809.4,139.61,162.26,1.0094,NaN,0.021275,NaN
2019-01-02,1.0073,3873.8,152.95,169.19,1.0704,NaN,0.021465,NaN
2019-01-03,1.0054,3780.1,146.94,158.99,0.8521,NaN,0.020387,NaN
2019-01-04,1.0066,3802.7,152.86,158.38,0.9652,NaN,0.021190,NaN
2019-01-05,1.0058,3785.4,153.49,157.38,0.9702,NaN,0.020791,NaN
...,...,...,...,...,...,...,...,...
2021-12-27,1.0003,50678.2,4036.24,465.10,0.9995,NaN,0.079048,90.90
2021-12-28,1.0001,47545.2,3791.69,438.50,0.9997,NaN,0.075003,85.09
2021-12-29,1.0001,46461.7,3627.93,429.70,0.9999,NaN,0.079848,84.15
2021-12-30,1.0000,47123.3,3709.57,431.70,0.9997,NaN,0.078377,84.76


# Stilized Facts: